In [ ]:
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/AOLMERobots.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/color_threshold_sliders_image.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/my_image.png
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/thresholds2.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/Python-All-Sessions//ColorMap.png

In [2]:
#@title Color threshold{ run: "auto" }
%matplotlib inline
import cv2
import AOLMERobots as gopi
from thresholds2 import *
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
 
 
image = cv2.imread('ColorMap.png')
height, width = image.shape[:2]
image= cv2.resize(image, (int(320/height*width), 320))
 
thr_obj = threshold(image)
 
 
 
def DisplayData(redLow, redHigh, greenLow, greenHigh, blueLow, blueHigh):
 
  img = image.copy()
 
  # Red component processing:
  thr_obj.sel_color_comp ('red')
  RedRange =  thr_obj.ThreshRange (redLow , redHigh)   
 
  # Green component processing:
  thr_obj.sel_color_comp ('green')
  GreenRange =  thr_obj.ThreshRange (greenLow , greenHigh)  
        
  # Blue component processing:
  thr_obj.sel_color_comp ('blue')
  BlueRange =  thr_obj.ThreshRange (blueLow , blueHigh)  
 
  # Get middle-RGB combination
  comb_image, comb_allTh= thr_combination(img, BlueRange,GreenRange, RedRange).return_result()   
  area = sum(comb_allTh)
  
  skip = 120
  # Generate the second window, including 9 single color images, one combination image, and text
  white = np.zeros((image.shape[0], skip), np.uint8)
  white[:] = (255)
  white_3_channel = cv2.merge((white, white, white))
 
  comb_allTh_3_channel = cv2.merge((comb_allTh*255, comb_allTh*255, comb_allTh*255)) 
  row_1 = np.hstack((img, white_3_channel))
  row_2 = np.hstack((row_1, comb_allTh_3_channel))
  row_3 = np.hstack((row_2, white_3_channel))
  row_4 = np.hstack((row_3, comb_image))
  


  white_title = np.zeros((50, row_4.shape[1]), np.uint8)
  white_title[:] = (255)
  white_title_3_channel = cv2.merge((white_title, white_title, white_title))
  
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(white_title_3_channel, 'Input', (120,40), font, 0.7, (0,0,0), 2, 0)
  cv2.putText(white_title_3_channel, 'Binary Image', (90+image.shape[0]+skip,40), font, 0.7, (0,0,0), 2, 0)
  cv2.putText(white_title_3_channel, 'Output', (120+image.shape[0]*2+skip*2,40), font, 0.7, (0,0,0), 2, 0)
  all = np.vstack((white_title_3_channel, row_4))

  cv2_imshow(all)


  rgb_values = [blueLow, blueHigh, greenLow, greenHigh, redLow, redHigh]
  
  # Show histograms for RGB color
  gopi.show_RGB_hist(img, rgb_values)
  print('The detected color region has {} pixels'.format(int(area)))

  #print(redLow,'<= Red   <= ', redHigh)
  #print(greenLow,'<= Green <= ', greenHigh)
  #print(blueLow,'<= Blue  <= ', blueHigh)
 
interactive_plot = interactive(DisplayData, 
                               redLow   = widgets.IntSlider(min=0, max=255, step=5, value=16),
                               redHigh  = widgets.IntSlider(min=0, max=255, step=5, value=156),
                               greenLow = widgets.IntSlider(min=0, max=255, step=5, value=30),
                               greenHigh= widgets.IntSlider(min=0, max=255, step=5, value=132),
                               blueLow  = widgets.IntSlider(min=0, max=255, step=5, value=24),
                               blueHigh = widgets.IntSlider(min=0, max=255, step=5, value=180))
interactive_plot

interactive(children=(IntSlider(value=16, description='redLow', max=255, step=5), IntSlider(value=156, descrip…